<a href="https://colab.research.google.com/github/Shruti10-cpu/Google-earth-/blob/main/Practical_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap.geemap as map
import ee
ee.Authenticate()
ee.Initialize(project ='mshruti724')
m = map.Map()

In [2]:
Maharashtra = ee.FeatureCollection('projects/mshruti724/assets/Maharashtra_shapefile')
m.addLayer(Maharashtra,{},'Maharashtra')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
region = ee.Geometry.Polygon(
    [[[74.060547, 7.740675],
     [74.060547, 14.065392],
     [82.964844, 14.065392],
     [82.964844, 7.740675],
     [74.060547, 7.740675]]],
    None,
    False)

In [4]:
ndvi = ee.ImageCollection("MODIS/061/MOD13A2").select('NDVI')

In [5]:
ndvi_TN = ndvi.map(lambda img: img.set('doy',ee.Date(img.get('system:time_start')).getRelative('day', 'year')));
distinctDOY = ndvi_TN.filterDate('2023-02-18', '2024-02-18');

In [6]:
filter = ee.Filter.equals(leftField ='doy',rightField = 'doy')
join = ee.Join.saveAll('doy_matches')
joincol = ee.ImageCollection(join.apply(distinctDOY,ndvi_TN,filter))

In [9]:
composite = joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()).set('doy',ee.Number(img.get('doy'))))

In [10]:
vis_para = {
  'min': 0,
  'max': 9000,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
};

In [11]:
rgbvis= composite.map(lambda img: img.visualize(bands = ['NDVI_mean'],**vis_para).clip(Maharashtra))

In [12]:
gifParams = {
  'region': region,
  'dimensions': 600,
  'crs': 'EPSG:4326',
  'framesPerSecond': 10,
  'format':'gif',
}

In [13]:
print(rgbvis.getVideoThumbURL(gifParams))

https://earthengine.googleapis.com/v1/projects/mshruti724/videoThumbnails/c7a1832779f2600ad88ef13b0c34685f-95cf7ab8b391fd09d00bd8d8dec94180:getPixels
